<a href="https://colab.research.google.com/github/satishkhanna/Challenges_in_Machine_Learning_project/blob/master/quickstarts/Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook contains examples of how to write and run your first prompts with the Gemini API.

## Learn more

There's lots more to learn!

* For more fun prompts, check out [Market a Jetpack](https://github.com/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb).
* Check out the [safety quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb) next to learn about the Gemini API's configurable safety settings, and what to do if your prompt is blocked.
* For lots more details on using the Python SDK, check out this [detailed quickstart](https://ai.google.dev/tutorials/python_quickstart).

In [2]:
import pandas as pd
import spacy
import numpy as np
import nltk
import openpyxl

In [50]:
df = pd.read_csv("/content/complaints.csv")

In [51]:
nlp = spacy.load ('en_core_web_sm')
doc = nlp(df.Sub_issue.iloc[0])

In [56]:
tokens = []
lemma = []
pos = []

for doc in nlp.pipe (df['Sub_issue'].astype('unicode').values, batch_size = 50):
  if doc.is_parsed:
    tokens.append([n.text for n in doc])
    lemma.append([n.lemma_ for n in doc])
    pos.append([n.pos_ for n in doc])
  else:
    tokens.append(None)
    lemma.append(None)
    pos.append(None)

df['issue_tokens'] = tokens
df['issue_lemma'] = lemma
df['issue_pos'] = pos

<ipython-input-56-2220fc4f665a>:6: DeprecationWarning: [W107] The property `Doc.is_parsed` is deprecated. Use `Doc.has_annotation("DEP")` instead.
  if doc.is_parsed:


In [58]:
def to_doc(words:tuple) -> spacy.tokens.Doc:
  return nlp(' '.join(words))

def remove_stops(doc) -> list:
  return [token.text for token in doc if not token.is_stop]

In [60]:
docs = list(map(to_doc, df.issue_lemma))
df['removed_stops'] = list(map(remove_stops, docs))

In [64]:
import re
#Remove punctuation
df['removed_stops_proces'] = df['removed_stops'].map(lambda x: re.sub("[,\.!?]","",str(x)))
#convert to lower
df['removed_stops_proces'] = df['removed_stops_proces'].map(lambda x:x.lower())

In [66]:
df['removed_stops_proces'] = df['removed_stops_proces'].str.replace("'",'')

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation